In [74]:
import json
import requests
import datetime
import pytz

In [103]:
# get amz-auth
url = "https://cognito-identity.us-east-1.amazonaws.com/"
payload = "{\"IdentityId\":\"us-east-1:5cb98804-4a0e-4004-aeae-f8feb41b7be5\"}"
# now = datetime.datetime.now(pytz.timezone('US/Eastern'))
now = datetime.datetime.utcnow()
headers = {
  'Referer': 'https://www.goodreads.com/',
  'X-Amz-User-Agent': 'aws-sdk-js/2.711.0 callback',
  'Content-Type': 'application/x-amz-json-1.1',
  'X-Amz-Target': 'AWSCognitoIdentityService.GetCredentialsForIdentity',
  'X-Amz-Content-Sha256': '6030ca7c1f3393246848c0323a5e26fea5b3f98543b92a2e409fdf030aba5245',
  'Origin': 'https://www.goodreads.com',
  'Connection': 'keep-alive',
  'Sec-Fetch-Mode': 'cors'
}
response = requests.request("POST", url, headers=headers, data=payload)
auth = json.loads(response.text)["Credentials"]
# auth

In [102]:
# get reviews
url = "https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql"
payload = "{\"operationName\":\"getReviews\",\"variables\":{\"filters\":{\"resourceType\":\"WORK\",\"resourceId\":\"kca://work/amzn1.gr.work.v1.jY9lyLQNtOx-St1AGeHgBA\"},\"pagination\":{\"limit\":30}},\"query\":\"query getReviews($filters: BookReviewsFilterInput!, $pagination: PaginationInput) {\\n  getReviews(filters: $filters, pagination: $pagination) {\\n    ...BookReviewsFragment\\n    __typename\\n  }\\n}\\n\\nfragment BookReviewsFragment on BookReviewsConnection {\\n  totalCount\\n  edges {\\n    node {\\n      ...ReviewCardFragment\\n      __typename\\n    }\\n    __typename\\n  }\\n  pageInfo {\\n    prevPageToken\\n    nextPageToken\\n    __typename\\n  }\\n  __typename\\n}\\n\\nfragment ReviewCardFragment on Review {\\n  __typename\\n  id\\n  creator {\\n    ...ReviewerProfileFragment\\n    __typename\\n  }\\n  recommendFor\\n  updatedAt\\n  createdAt\\n  spoilerStatus\\n  lastRevisionAt\\n  text\\n  rating\\n  shelving {\\n    shelf {\\n      name\\n      webUrl\\n      __typename\\n    }\\n    taggings {\\n      tag {\\n        name\\n        webUrl\\n        __typename\\n      }\\n      __typename\\n    }\\n    webUrl\\n    __typename\\n  }\\n  likeCount\\n  viewerHasLiked\\n  commentCount\\n}\\n\\nfragment ReviewerProfileFragment on User {\\n  id: legacyId\\n  imageUrlSquare\\n  isAuthor\\n  ...SocialUserFragment\\n  textReviewsCount\\n  viewerRelationshipStatus {\\n    isBlockedByViewer\\n    __typename\\n  }\\n  name\\n  webUrl\\n  contributor {\\n    id\\n    works {\\n      totalCount\\n      __typename\\n    }\\n    __typename\\n  }\\n  __typename\\n}\\n\\nfragment SocialUserFragment on User {\\n  viewerRelationshipStatus {\\n    isFollowing\\n    isFriend\\n    __typename\\n  }\\n  followersCount\\n  __typename\\n}\\n\"}"
amz_date = now.strftime("%Y%m%d")
amz_time = now.strftime("%H%M%S")
headers = {
  'Accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br',
  'Referer': 'https://www.goodreads.com/',
  'content-type': 'application/json; charset=UTF-8',
  'x-amz-date': f"{amz_date}T{amz_time}Z",
  'X-Amz-Security-Token': auth["SessionToken"],
  'Authorization': f'AWS4-HMAC-SHA256 Credential={amz_date}T{amz_time}Z/{amz_date}/us-east-1/appsync/aws4_request, SignedHeaders=accept;content-type;host;x-amz-date;x-amz-security-token, Signature={auth["SecretKey"]}',
  'x-amz-user-agent': 'aws-amplify/2.0.2',
  'Origin': 'https://www.goodreads.com',
  'Connection': 'keep-alive',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'cross-site',
  'TE': 'trailers'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

{
  "errors" : [ {
    "errorType" : "UnrecognizedClientException",
    "message" : "The security token included in the request is invalid."
  } ]
}


In [101]:
print(json.loads(response.text)["errors"][0]["message"])

The request signature we calculated does not match the signature you provided. Check your AWS Secret Access Key and signing method. Consult the service documentation for details.

The Canonical String for this request should have been
'POST
/graphql

accept:*/*
content-type:application/json; charset=UTF-8
host:kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com
x-amz-date:20230421T194759Z
x-amz-security-token:IQoJb3JpZ2luX2VjELT//////////wEaCXVzLWVhc3QtMSJIMEYCIQCqbr4jn6CcpdqELq9yfOel46cy80VlWE2s8RhSaCz8pgIhANFyVF6IkuDA0fHylJVlMVQWqw9AXUI1GUyg/rjsFpZTKpAGCK3//////////wEQAhoMNTE4MDgzOTIyMTI3Igw7/PhRjMBRjz2tUKQq5AX6rbWfQfCCFNYAgXrUOiCIO4B6i1FFmFGf57IZ0o5xmbJzzh3f2FpUzHOGE/p/Zn78/3KxnkDzy/P4H3CQ/vQm1FGkHdt/tCH+OKu1DFpXM03HRslVhtdIVjhWmoz5Ivk2QYNBLty63RrKTXRU+fThMLsBvfBOseSZT0itgLLcOjenM5l3XmEqk/1uyYCWFBg3RHVXwU3iDAvh7SOSp2avdccp3V7C4aEjZcsVWSRptYhnU4f1evB+CnUfhlM48fQmpZHYWrZdllmJB8DkkG7Eo4PzQkFHsbrtUzyTO14qW8KRbV3psx7d7HJANMS60gCxpQt3+qcdiTwHohHApgPM+Re43avzfmiUQZYWhXwC/pNaVBcv

In [78]:
import datetime

In [72]:
import pytz
now = datetime.datetime.now(pytz.timezone('US/Eastern'))
amz_date = now.strftime("%Y%m%d")
amz_time = now.strftime("%H%M%S")
print(amz_date)
print(amz_time)

20230421
153453


In [85]:
print(datetime.datetime.fromtimestamp(auth["Expiration"]).strftime("%Y-%m-%d_%H:%M:%S"))
print(now.strftime("%Y-%m-%d_%H:%M:%S"))

2023-04-21_15:39:28
2023-04-21_19:39:28


In [ ]:
20230421T153540Z
20230421T193043Z

In [97]:
headers

{'Accept': '*/*',
 'Accept-Language': 'en-US,en;q=0.5',
 'Accept-Encoding': 'gzip, deflate, br',
 'Referer': 'https://www.goodreads.com/',
 'content-type': 'application/json; charset=UTF-8',
 'x-amz-date': '20230421T193928Z',
 'X-Amz-Security-Token': 'IQoJb3JpZ2luX2VjELT//////////wEaCXVzLWVhc3QtMSJHMEUCIHLWUU0/PYWHRP0Wn64MSF8ND9ZAXHstvheZjR/g9MzwAiEAn0AEvPwUkv+gDsxcauPNZ/cXipvA29clhr43kLW1hRcqkAYIrf//////////ARACGgw1MTgwODM5MjIxMjciDK0ZsE4cgkIgWaVVPSrkBejjiTmWE8Job4nDA3gDUgseLlrloCNk7VTGZG9bVX2ZsuZgAl5pyYNPC2s1xFysrEu2tZhgQDh+bCpZDy8EjI9M28uucjWQh5AMnrU+jmywmuoQrmcR3lKB53/QHzRMFgxA0g4qv2KKt0pXKUow4e+KwNNzcvqpTZntr1cGy4o5gNEPYxESn2W0TOxza5QwW9Biy/3x/qvzaTBjJGK6gmpedRNhrKvi7IavIDQHHUjPNoJkXAjX6zZgN3FYZiBwBMWx5je1MxzYo7wPX+tpc5TvD++m02+geeq+7cQonsyPenlJr0ZhqAH0IIvQtauCJ85I/xOYRSyR3Ill7pFFu23FN/rG7GVvSA+hNMcMdQWO/XHDtq+HsAfjjc/a8NOulIfbrmM3K0ie54TpE2bhNznfI4DjBp/jm5Hb8P5b5RorM/jv4ixNFH0HVzPuZRfHXAcReLVhL8mzu72NkxZ4jlWFN98SdYMCLA7EipPDiwC5fH51FN7Cba4f9MktRFflLUp9bATs74zxFieoZrM1SfrtiUYwV7Qf

In [17]:
import requests
url = "https://www.goodreads.com/book/show/112077.The_Game_of_Kings"
response = requests.get(url)

In [22]:
vars(response).keys()

dict_keys(['_content', '_content_consumed', '_next', 'status_code', 'headers', 'raw', 'url', 'encoding', 'history', 'reason', 'cookies', 'elapsed', 'request', 'connection'])

In [28]:
response.connection